In [26]:
from common import *
%matplotlib inline
canvas(9, 6)

In [41]:
from sklearn.linear_model import LinearRegression as LR

def do_lr(X, Y, W):
    lr_list = []
    # assert the columns of the input can be indexed by range(0, 3023)
    for k in Y.columns:
        x, y, w = X.loc[:, (ss, k)], Y[k], W[k]
        if w.sum() == 0:
            lr_list += [None]
            continue
        lr = LR(fit_intercept=False)
        # don't forget x1 and x2 have NaNs for valid y. try setting W=0 for when y and W does not exist and then filling in NaNs
        lr = lr.fit(x, y, w)
        lr_list += [lr]
    return lr_list

In [51]:
Wraw = make_stock_wide(dh.Weight)
_demean = lambda x: x - x.wmean(Wraw)
Y = make_stock_wide(dh.y).fillna(0)
W = make_stock_wide(dh.Weight).fillna(0)

ifilt = index_nvdf(dh, 246)
sfilt = stock_nvdf(dh, 246)
dhlog = np.log(dh.replace(0, np.exp(-200)))

_day = dh[['Day']]
_day.columns += '.scl'
Xfd = ((_day - _day.min()) / (_day.max() - _day.min())).pipe(make_stock_wide).pipe(_demean).fillna(0)

Xpos = make_stock_wide(dhlog[['x2']] - dhlog[['x3']] - dhlog[['x4']]).wpct(W).fillna(0.5) * 2 - 1
Xneg = make_stock_wide(dh[['x3']] + dh[['x4']] + dh[['x5']]).wpct(W).fillna(0.5) * 2 - 1

/home/ec2-user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in log
  


In [52]:
with open(top_dir + 'data/chance/days_order.txt', 'r') as file:
    days = pd.Series([int(x) for x in file.read().splitlines()], name='Day')
n_days = len(days)
days_a, days_b, days_c = days[:n_days // 3], days[n_days // 3:(2 * n_days) // 3], days[(2 * n_days) // 3:]
days_z = dg.Day.unique()

In [53]:
tr, cv, ep, sm = Y.index.isin(days_a), Y.index.isin(days_b), Y.index.isin(days_c), Y.index.isin(days_z)

In [54]:
X = Xfd # <==============
sub = np.concatenate([sfilt] * X.columns.levels[0].shape[0]) # <===============

In [55]:
Xtr, Ytr, Wtr = var_tr = [item[tr].T[sub].T for item in [X, Y, W]]
Xcv, Ycv, Wcv = var_cv = [item[cv].T[sub].T for item in [X, Y, W]]
Xep, Yep, Wep = var_ep = [item[ep].T[sub].T for item in [X, Y, W]]
Xsm, Ysm, Wsm = var_sm = [item[sm].T[sub].T for item in [X, Y, W]]

In [56]:
ywmean = pd.concat([Ytr, Ycv]).wmean(pd.concat([Wtr, Wcv]))

In [57]:
lr_list = do_lr(Xtr, Ytr - ywmean, Wtr)
model = np.load(top_dir + 'data/progress/niceee.npy')

In [58]:
XX, YY, WW = var_cv # <=============

In [66]:
YY_ = np.stack([(lr.predict(XX.loc[:, (ss, k)]) if lr is not None else np.zeros(YY.shape[0]))
               for lr, k in zip(lr_list, YY.columns)], axis=1)
YY_ = YY_ + YY * 0 + ywmean
#
YYo_ = np.stack([XX.loc[:, (ss, k)] @ beta for beta, k in zip(model, YY.columns)], axis=1)
YYo_ = YYo_ + YY * 0 + ywmean

In [67]:
(
    ((YY_ - YY) ** 2 * WW).sum().sum() / ((YY ** 2 * WW).sum().sum()),
    ((YYo_ - YY) ** 2 * WW).sum().sum() / ((YY ** 2 * WW).sum().sum())
)

(0.72840278266034386, 9.5059765438984947)

### stuff's going down

In [101]:
Ypos = Ycv_

In [43]:
def get_ans(i):
    row = dh.loc[i]
    which = filt.loc[i]
    return ypos.loc[row.Day, row.Stock] if which else yneg.loc[row.Day, row.Stock]
yy = pd.Series(np.array([get_ans(i) for i in dh.index]), index=dh.index)

KeyboardInterrupt: 